In [1]:
import json
import random
import pandas as pd
from tqdm import tqdm

In [3]:
data = json.load(open("../data/lang=en-data=full.json", "r"))
data_en = json.load(open("../data/lang=en-data=split_en.json", "r"))

In [4]:
data_ja = [i for i in data if i not in data_en]

In [6]:
for i in data_en:

4946

In [2]:
def split_list_by_ratio(lst, ratios):
    total = len(lst)
    sizes = [round(r * total) for r in ratios]

    # Adjust sizes to make sure the total sum is equal to the list length
    size_difference = total - sum(sizes)
    sizes[-1] += size_difference

    # Split the list
    chunks = []
    start = 0
    for size in sizes:
        chunks.append(lst[start:start + size])
        start += size

    return chunks

In [3]:
data_en = json.load(open("../data/lang=en-data=split_en.json", "r"))
data_ja = json.load(open("../data/lang=ja-data=split_ja.json", "r"))

In [4]:
data_ja[0]

{'text': '革新的な都市開発プロジェクトの成功した実行を目撃することは、以前は荒れ果てていた場所が、現代的なインフラストラクチャと活気ある公共スペースを備えた繁栄するコミュニティ拠点に変わったことで私に喜びをもたらしました。',
 'choice': '喜び',
 'explanation': 'その文章は都市景観のポジティブな変化に対する感情的な反応を捉え、喜びの感情を発散しています。"成功した実行"、"革新的"、"繁栄するコミュニティハブ"、"活気ある公共スペース"などの言葉の使用は、都市計画や開発の活性化と進歩によって引き起こされる感情的な浮かび上がりや幸福感を伝えています。',
 'id': 'generated'}

In [11]:
import re

def find_english_phrases(text):
    # Regular expression pattern for connected English phrases
    pattern = r'[a-zA-Z]+(?:\s+[a-zA-Z]+)*'

    # Find all non-overlapping matches of the pattern
    english_phrases = re.findall(pattern, text)

    return english_phrases

# Example usage
mixed_text = 'これはテ"manipulation"ストです with some English words。英語がmixedしています。'
english_phrases = find_english_phrases(mixed_text)
print("English phrases found:", english_phrases)

English phrases found: ['manipulation', 'with some English words', 'mixed']


In [12]:
from deep_translator import GoogleTranslator

# Use any translator you like, in this example GoogleTranslator
translator = GoogleTranslator(source='auto', target='ja')  # output -> Weiter so, du bist großartig

In [64]:
data_ja = pd.read_csv("../data/training_lang=ja-data=split_ja.csv").to_dict('records')

ignore_words = ["COVID", "STEM", "3D"]

for item in tqdm(data_ja):
    text_en_words = find_english_phrases(item["input"])
    if text_en_words != []:
        for phrase in text_en_words:
            if phrase not in ignore_words:
                translated_phrase = translator.translate(phrase)
                item["input"] = item["input"].replace(phrase, translated_phrase)

    explanation_en_words = find_english_phrases(item["output"])

    if explanation_en_words != []:
        for phrase in explanation_en_words:
            if phrase not in ignore_words:
                translated_phrase = translator.translate(phrase)
                item["output"] = item["output"].replace(phrase, translated_phrase)

  0%|          | 0/1845 [00:00<?, ?it/s]

100%|██████████| 1845/1845 [04:12<00:00,  7.31it/s]


In [66]:
pd.DataFrame(data_ja).to_csv("../data/training_lang=ja-data=split_ja_corrected.csv", index=False)

In [23]:
[i for i in set(english_words) if len(i) >= 3]

['disregard for the well',
 'prospect',
 'feeling drained',
 'disgusting',
 'peruse',
 'depleted',
 'blown away',
 'Astrophysics',
 'sacs',
 'dynamic',
 'unfairness',
 'absolutely infuriating',
 'Here is the translation from English to Japanese',
 'OCD',
 're considering asking only vaccinated people to visit',
 'vibrant',
 'apprehension',
 'seething anger',
 'NASA',
 'ominous',
 'prepares',
 'retirement planning options',
 'unsportsmanlike conduct',
 'loss',
 'perfectly seared',
 'promised advancements',
 'disregard for negative impact',
 'reckless construction',
 'frustration',
 'disillusionment',
 'mushy',
 'sentiment',
 'Groundbreaking air purification technology',
 'insulting',
 'which ultimately cost our team the championship title despite months of hard work and training',
 'the colorful walls and toys devoid of the youthful energy',
 'proud',
 'privacy breaches',
 'uplift',
 'promise',
 'shattered',
 't wait to go back',
 'potential for significant growth',
 'nauseated',
 'eage

In [15]:
for i in set(english_words):
    if len(i) >= 4:
        print(i)

nest
cloud
prospect
plan
growing
scientist
Space
artistic
disgusting
feel
next
peruse
quarantine
depleted
cases
tantalizing
Astrophysics
sacs
taken
dynamic
unknown
unfairness
energy
embracing
spending
mental
communities
monumental
state
chewed
time
phase
vibrant
apprehension
awaiting
scene
NASA
prepares
ominous
brought
beautiful
unwavering
treating
translation
maintaining
loss
advancements
awaited
enjoy
blown
rewarding
frustration
disillusionment
pick
came
races
inequality
mushy
ethical
sentiment
insulting
Exploration
work
mold
people
cleaner
proud
uplift
promise
array
shattered
friend
waves
nauseated
drained
vaccination
changes
engage
They
After
asking
sizzling
infestation
passionate
palpable
enraging
disrespected
tedious
meal
reeling
enrages
hobbies
recoil
funding
feelings
once
turn
disturbed
diving
inconceivable
easily
reputation
writer
live
creaked
Japanese
excessively
teachers
desparate
stomach
secure
daunting
dilution
Melancholic
bated
malfunctions
rolled
manipulation
immense
spe

In [5]:
data_en_concatenated = list()

for item in data_en:
    data_en_concatenated.append({"input": f"[TEXT]: {item['text']} [EMOTION]: {item['choice']}", "output": f"[RATIONALE]: {item['explanation']}"})

data_ja_concatenated = list()

for item in data_ja:
    data_ja_concatenated.append({"input": f"[文章]: {item['text']} [感情]: {item['choice']}", "output": f"[説明]: {item['explanation']}"})

In [6]:
# Example usage
ratios = [0.75, 0.1, 0.15]
data_en_train, data_en_val, data_en_test = split_list_by_ratio(data_en_concatenated, ratios)
data_ja_train, data_ja_val, data_ja_test = split_list_by_ratio(data_ja_concatenated, ratios)

In [13]:
ignore_words = ["COVID", "STEM", "3D"]

for item in tqdm(data_ja_test):
    text_en_words = find_english_phrases(item["input"])
    if text_en_words != []:
        for phrase in text_en_words:
            if phrase not in ignore_words:
                translated_phrase = translator.translate(phrase)
                item["input"] = item["input"].replace(phrase, translated_phrase)

    explanation_en_words = find_english_phrases(item["output"])

    if explanation_en_words != []:
        for phrase in explanation_en_words:
            if phrase not in ignore_words:
                translated_phrase = translator.translate(phrase)
                item["output"] = item["output"].replace(phrase, translated_phrase)

100%|██████████| 369/369 [00:50<00:00,  7.28it/s]


In [14]:
ignore_words = ["COVID", "STEM", "3D"]

for item in tqdm(data_ja_val):
    text_en_words = find_english_phrases(item["input"])
    if text_en_words != []:
        for phrase in text_en_words:
            if phrase not in ignore_words:
                translated_phrase = translator.translate(phrase)
                item["input"] = item["input"].replace(phrase, translated_phrase)

    explanation_en_words = find_english_phrases(item["output"])

    if explanation_en_words != []:
        for phrase in explanation_en_words:
            if phrase not in ignore_words:
                translated_phrase = translator.translate(phrase)
                item["output"] = item["output"].replace(phrase, translated_phrase)

100%|██████████| 246/246 [00:39<00:00,  6.24it/s]


In [9]:
pd.DataFrame(data_en_val).to_csv("val_lang=en-data=split_en.csv", index=False)
pd.DataFrame(data_en_test).to_csv("test_lang=en-data=split_en.csv", index=False)


In [6]:
pd.DataFrame(data_ja_val).to_csv("val_lang=ja-data=split_ja.csv", index=False)
pd.DataFrame(data_ja_test).to_csv("test_lang=ja-data=split_ja.csv", index=False)